# The following loads your model and defines the dataset to use

In [1]:
import torch
import data_loader
import numpy as np
import calculate_log as callog
import models
import os
import lib_generation

from torchvision import transforms
from torch.autograd import Variable

BATCH_SIZE = 200
DATAROOT='./data'
NUM_CLASSES = 10
GPU = 0
DATASET = 'cifar10'
NET_TYPE = 'resnet'

# set the path to pre-trained model and output
pre_trained_net = './pre_trained/' + NET_TYPE + '_' + DATASET + '.pth'
torch.cuda.manual_seed(0)
torch.cuda.set_device(GPU)

# load networks
if NET_TYPE == 'densenet':
    if DATASET == 'svhn':
        model = models.DenseNet3(100, int(NUM_CLASSES))
        model.load_state_dict(torch.load(pre_trained_net, map_location = "cuda:" + str(GPU)))
    else:
        model = torch.load(pre_trained_net, map_location = "cuda:" + str(GPU))
    in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((125.3/255, 123.0/255, 113.9/255), (63.0/255, 62.1/255.0, 66.7/255.0)),])
elif NET_TYPE == 'resnet':
    model = models.ResNet34(num_c=NUM_CLASSES)
    model.load_state_dict(torch.load(pre_trained_net, map_location = "cuda:" + str(GPU)))
    in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
model.cuda()
print('load model: ' + NET_TYPE)

model.eval()

# load dataset
print('load target data: ', DATASET)
train_loader, test_loader = data_loader.getTargetDataSet(DATASET, BATCH_SIZE, in_transform, DATAROOT)

load model: resnet
load target data:  cifar10
Files already downloaded and verified
Files already downloaded and verified


# Print out the structure of the network. 

It's not required to do this, but gives an idea of how to target specific layers that you want the features from.

You can call `layer.children()` on layers with nested children, like the "Sequential" blocks to access their child layers

In [2]:
for layer in model.children():
    print(layer)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )

# Get intermediate layer activations
## Basic example

Easiest way I found so far to get intermediate layer outputs is to register "forward hooks" that basically just appends these outputs to a list. You add different hooks to different layers for efficiency and to prevent iterating over the whole dataset. The following example adds a hook to just the first Conv2D layer

In [3]:
#####
outputs= []
def hook(module, input, output):
    activations = output.detach().cpu()  # Off GPU
    activations = activations.data.numpy()  # To numpy
    outputs.append(activations)

# Add the hook to the first layer (Conv2D)
layer = list(model.children())[0]
print(layer)
conv2d_hook = layer.register_forward_hook(hook)

# Note, if you add the hook multiple times, you'll get the output appended multiple times for each forward pass

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [4]:
outputs = []

# For inlier dataset, get activations at each layer
for train_x, train_y in train_loader:
    # Run all data through the model so it hits the forward hook
    model(train_x.cuda())

In [5]:
print(len(outputs))
print(outputs[0].shape)

250
(200, 64, 32, 32)


Theres 250 batches of size 200. The first layer has dimensions 64x32x32.

We can flatten the features over the dataset like so:

In [6]:
outputs = np.concatenate(outputs, axis=0)
print(outputs.shape)

(50000, 64, 32, 32)


## More complex example

Say you want the activation layers for each conv1 layer in the BasicBlocks of the model (i.e. refering to the printed model structure output above):

In [7]:
# Detach the old hook
conv2d_hook.remove()

In [8]:
# There's probably a more efficient way to do it, but you need to get a reference to 
#    each layer and add the forward hook to it
model_layers = list(model.children())
sequential_layers = model_layers[2:6]

all_basic_blocks = []
for l in sequential_layers:
    all_basic_blocks += list(l.children())
    
all_conv1_in_basic_blocks = [list(l.children())[0] for l in all_basic_blocks]
print(all_conv1_in_basic_blocks)

[Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False), Conv2d(256, 256, kernel_size=(3, 3

In [9]:
print(len(all_conv1_in_basic_blocks))

16


In [10]:
# Attach a hook to each of these layers
conv1_outputs= []

for i, conv1 in enumerate(all_conv1_in_basic_blocks):
    conv1_outputs.append([])
    
    def conv1_hook(module, input, output):
        activations = output.detach().cpu()  # Off GPU
        activations = activations.data.numpy()  # To numpy
        conv1_outputs[0].append(activations)
    
    conv1.register_forward_hook(conv1_hook)

In [11]:
# Then you could run through the data to get all the features.
# Skip doing this here because it's a huge amount of memory to save all the features from 16 layers over 50000 samples

# For inlier dataset, get activations at each layer
# for train_x, train_y in train_loader:
#     # Run all data through the model so it hits the forward hook
#     model(train_x.cuda())